# ДЗ Lite. GPT Professional. Нейро-продажник (урок 1):

Скорректируйте модель - стилиста так, чтобы менеджер по продажам отвечал на вопросы клиента как какой-нибудь известный персонаж (вымышленный или реальный)


# Системные функции

In [1]:
!pip  install openai >/dev/null
#!pip -q install oauth2client >/dev/null

In [2]:
#@title Грузим библиотеки

import ipywidgets as widgets
import os
from oauth2client.service_account import ServiceAccountCredentials

#from google.colab import output
import openai



In [3]:

#@title Определим класс для подсветки вывода разных моделей разными цветами
class bcolors:
  # https://en.wikipedia.org/wiki/ANSI_escape_code#Colors
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    GREEN = '\033[32m'
    RED = '\033[31m'
    BGGREEN = "\033[102m"
    BGYELLOW = "\033[103m"
    BGCYAN = "\033[106m"
    BGMAGENTA = "\033[105m"
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [4]:
# @title Определяем функции python

# Функция удаления перехода на новую строку
def remove_newlines(text):
    cleaned_string = text.replace('\n', ' ')
    return cleaned_string

#фунция добавления переходов на новую строку для удобства чтения
# здесь в параметр text подается текст без переносов строки
# функция добавит переходы на новую строку при достижении max_len символов
def insert_newlines(text: str, max_len: int = 160) -> str:
      words = text.split()
      lines = []
      current_line = ""
      for word in words:
          if len(current_line + " " + word) > max_len:
              lines.append(current_line)
              current_line = ""
          current_line += f' {word}'
      lines.append(current_line)
      return "\n".join(lines)

In [5]:
#@title Задаем константы
gpt_4o = 'gpt-4o'
gpt_4o_mini = 'gpt-4o-mini'
MODEL = gpt_4o_mini

temp = 0.1
verbose = 1

In [6]:

#@title  Используем секретный ключ в колабе
from google.colab import userdata
key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = key


In [7]:
#@title Модель - стилист (отвечает как известный персонаж)
def stilizator_answer_as_character(name, character, answers_content, temp=0.7, verbose=0, model=MODEL):
    """
    Преобразует сухой ответ менеджера в реплику, стилизованную под известного персонажа.

    Args:
        name (str): Имя модели (для логгирования)
        character (str): Имя персонажа (например: "Тони Старк", "Шерлок Холмс", "Матрица")
        answers_content (str): Исходный текст ответа менеджера
        temp (float): Температура генерации
        verbose (int): Уровень детализации лога
        model (str): Модель OpenAI
    """
    system_prompt = f"""Вы — менеджер по продажам, который отвечает клиенту, полностью перевоплощаясь в персонажа «{character}».
Ваши ответы должны:
- Сохранять суть и факты исходного ответа,
- Передавать манеру речи, интонацию, стиль и характерные фразы персонажа,
- Оставаться вежливыми и профессиональными (это всё ещё продажи!),
- Не выдумывать новые факты о продукте — только переформулировать исходное.

Если персонаж вымышленный — используйте его известные черты из фильмов, книг или сериалов.
"""

    user_prompt = f"""Преобразуйте следующий текст в реплику от лица персонажа «{character}»:

Исходный текст: {answers_content}

Ответ:"""

    if verbose:
        print('==================')
        print(f'{bcolors.BGCYAN}Текст для стилизации:{bcolors.ENDC}\n{answers_content}')
        print(f'{bcolors.BGYELLOW}Персонаж: {character}{bcolors.ENDC}')

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    completion = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp
    )

    result = completion.choices[0].message.content.strip()

    if verbose:
        print('\n==================')
        print(f'{completion.usage.total_tokens} total tokens used.')
        print('==================')
        print(f'{bcolors.BGMAGENTA}Ответ {name} (как {character}):{bcolors.ENDC}\n',
              f'{bcolors.HEADER}{result}{bcolors.ENDC}')

    return result

# Ваше решение

In [14]:
#@title Запускаем Стилист (как персонаж)
temperature_stilist = 0.7  #@param {type: "slider", min: 0.1, max: 1.0, step: 0.1}
verbose_stilist = "1"  #@param ["0", "1"]
character = "Магистр йода"  #@param {type: "string"}  # ← Меняйте на любого персонажа!
text_for_styling = "Спасибо за интерес! У нас есть три тарифа: Light, PRO и PRO MAX. PRO MAX включает 60 Zoom-консультаций с разработчиками и гарантию трудоустройства."  #@param {type: "string"}

output_stilist = stilizator_answer_as_character(
    name='Менеджер-персонаж',
    character=character,
    answers_content=text_for_styling,
    temp=temperature_stilist,
    verbose=int(verbose_stilist),
    model=MODEL
)

print("\n" + "="*80)
print("🎭 ФИНАЛЬНЫЙ ОТВЕТ МЕНЕДЖЕРА (как персонаж):")
print("="*80)
print(insert_newlines(remove_newlines(output_stilist), 90))

Текст для стилизации:
Спасибо за интерес! У нас есть три тарифа: Light, PRO и PRO MAX. PRO MAX включает 60 Zoom-консультаций с разработчиками и гарантию трудоустройства.
Персонаж: Магистр йода

273 total tokens used.
Ответ Менеджер-персонаж (как Магистр йода):
 Благодарю за интерес, я! Три тарифа у нас есть, да. Light, PRO и PRO MAX, это они. PRO MAX включает 60 консультаций с разработчиками, да, и гарантию трудоустройства, он имеет. Мудрое решение, это может быть!

🎭 ФИНАЛЬНЫЙ ОТВЕТ МЕНЕДЖЕРА (как персонаж):
 Благодарю за интерес, я! Три тарифа у нас есть, да. Light, PRO и PRO MAX, это они. PRO
 MAX включает 60 консультаций с разработчиками, да, и гарантию трудоустройства, он имеет.
 Мудрое решение, это может быть!




---



